In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator
to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn
rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, 
customer retention has now become even more important than customer acquisition.
For many incumbent operators, retaining high profitable customers is the number one business goal.
To reduce customer churn, telecom companies need to predict which customers are at high risk of churn.

In this project, you will analyse customer-level data of a leading telecom firm, build predictive models to identify 
customers at high risk of churn and identify the main indicators of churn.

Understanding and Defining Churn
There are two main models of payment in the telecom industry - postpaid (customers pay a monthly/annual bill after
using the services) and prepaid (customers pay/recharge with a certain amount in advance and then use the services).

In the postpaid model, when customers want to switch to another operator, they usually inform the existing operator 
to terminate the services, and you directly know that this is an instance of churn.

However, in the prepaid model, customers who want to switch to another network can simply stop using the services without 
any notice, and it is hard to know whether someone has actually churned or is simply not using the services temporarily 
(e.g. someone may be on a trip abroad for a month or two and then intend to resume using the services again).
 

Thus, churn prediction is usually more critical (and non-trivial) for prepaid customers, and the term ‘churn’ should be
defined carefully.  Also, prepaid is the most common model in India and southeast Asia, while postpaid is more common in
Europe in North America. This project is based on the Indian and Southeast Asian market.

 
Definitions of Churn
There are various ways to define churn, such as:
1. Revenue-based churn: 
    Customers who have not utilised any revenue-generating facilities such as mobile internet, outgoing calls, SMS etc. 
    over a given period of time. One could also use aggregate metrics such as ‘customers who have generated less than 
    INR 4 per month in total/average/median revenue’.

The main shortcoming of this definition is that there are customers who only receive calls/SMSes from their wage-earning 
counterparts, i.e. they don’t generate revenue but use the services. For example, many users in rural areas only receive 
calls from their wage-earning siblings in urban areas. 

2. Usage-based churn: Customers who have not done any usage, either incoming or outgoing - in terms of calls, internet etc.
    over a period of time.
    A potential shortcoming of this definition is that when the customer has stopped using the services for a while, 
    it may be too late to take any corrective actions to retain them. 
    For e.g., if you define churn based on a ‘two-months zero usage’ period, predicting churn could be useless since by
    that time the customer would have already switched to another operator.

<B>In this project, you will use the usage-based definition to define churn.</B>
 

3. High-value Churn: In the Indian and the southeast Asian market, approximately 80% of revenue comes from the top 20%
    customers (called high-value customers). Thus, if we can reduce churn of the high-value customers, we will be able to
    reduce significant revenue leakage.

In this project, you will define high-value customers based on a certain metric (mentioned later below) and predict churn 
only on high-value customers. 

Understanding the Business Objective and the Data
The dataset contains customer-level information for a span of four consecutive months - June, July, August and September.
The months are encoded as 6, 7, 8 and 9, respectively. 

The business objective is to predict the churn in the last (i.e. the ninth) month using the data (features) from the first
three months. To do this task well, understanding the typical customer behaviour during churn will be helpful.

Understanding Customer Behaviour During Churn
Customers usually do not decide to switch to another competitor instantly, but rather over a period of time
(this is especially applicable to high-value customers). In churn prediction, we assume that there are three phases of customer lifecycle :

The ‘good’ phase: In this phase, the customer is happy with the service and behaves as usual.

The ‘action’ phase: The customer experience starts to sore in this phase, for e.g. he/she gets a compelling offer from a  
    competitor, faces unjust charges, becomes unhappy with service quality etc. In this phase, the customer usually shows different behaviour than the ‘good’ months. Also, it is crucial to identify high-churn-risk customers in this phase, since some corrective actions can be taken at this point (such as matching the competitor’s offer/improving the service quality etc.)

The ‘churn’ phase: In this phase, the customer is said to have churned. You define churn based on this phase. 
    Also, it is important to note that at the time of prediction (i.e. the action months), this data is not available to you for prediction. Thus, after tagging churn as 1/0 based on this phase, you discard all data corresponding to this phase.

In this case, since you are working over a four-month window, the first two months are the ‘good’ phase, the third month is
the ‘action’ phase, while the fourth month is the ‘churn’ phase.

In [1]:
#Importing class & Libraries 
import pandas as pd, numpy as np

In [2]:
# Reading the dataset
data = pd.read_csv("telecom_churn_data.csv")

In [3]:
data.head()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.385,...,0,1.0,1.0,1.0,NaN,968,30.4,0.0,101.20,3.58
1,7001865778,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,34.047,...,0,NaN,1.0,1.0,NaN,1006,0.0,0.0,0.00,0.00
2,7001625959,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,167.690,...,0,NaN,NaN,NaN,1.0,1103,0.0,0.0,4.17,0.00
3,7001204172,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,221.338,...,0,NaN,NaN,NaN,NaN,2491,0.0,0.0,0.00,0.00
4,7000142493,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,261.636,...,0,0.0,NaN,NaN,NaN,1526,0.0,0.0,0.00,0.00


In [4]:
data.shape

(99999, 226)

In [5]:
data.info(verbose=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 226 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   mobile_number             int64  
 1   circle_id                 int64  
 2   loc_og_t2o_mou            float64
 3   std_og_t2o_mou            float64
 4   loc_ic_t2o_mou            float64
 5   last_date_of_month_6      object 
 6   last_date_of_month_7      object 
 7   last_date_of_month_8      object 
 8   last_date_of_month_9      object 
 9   arpu_6                    float64
 10  arpu_7                    float64
 11  arpu_8                    float64
 12  arpu_9                    float64
 13  onnet_mou_6               float64
 14  onnet_mou_7               float64
 15  onnet_mou_8               float64
 16  onnet_mou_9               float64
 17  offnet_mou_6              float64
 18  offnet_mou_7              float64
 19  offnet_mou_8              float64
 20  offnet_mou_9              f

In [18]:
#  unique values in each column of dataset
data.nunique().sort_values(ascending=False)

mobile_number       99999
arpu_6              85681
arpu_7              85308
arpu_8              83615
arpu_9              79937
                    ...  
std_ic_t2o_mou_8        1
std_ic_t2o_mou_9        1
std_og_t2c_mou_9        1
std_og_t2c_mou_8        1
std_ic_t2o_mou_6        1
Length: 226, dtype: int64

- <b>Inference
    - All the mobile numbers are unique and there is no duplicacy in mobile numbers.

In [6]:
data.describe()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,arpu_9,onnet_mou_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
count,9.999900e+04,99999.0,98981.0,98981.0,98981.0,99999.000000,99999.000000,99999.000000,99999.000000,96062.000000,...,99999.000000,25153.000000,25571.000000,26339.000000,25922.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000
mean,7.001207e+09,109.0,0.0,0.0,0.0,282.987358,278.536648,279.154731,261.645069,132.395875,...,0.084581,0.914404,0.908764,0.890808,0.860968,1219.854749,68.170248,66.839062,60.021204,3.299373
std,6.956694e+05,0.0,0.0,0.0,0.0,328.439770,338.156291,344.474791,341.998630,297.207406,...,0.650457,0.279772,0.287950,0.311885,0.345987,954.733842,267.580450,271.201856,253.938223,32.408353
min,7.000000e+09,109.0,0.0,0.0,0.0,-2258.709000,-2014.045000,-945.808000,-1899.505000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,180.000000,0.000000,0.000000,0.000000,0.000000
25%,7.000606e+09,109.0,0.0,0.0,0.0,93.411500,86.980500,84.126000,62.685000,7.380000,...,0.000000,1.000000,1.000000,1.000000,1.000000,467.000000,0.000000,0.000000,0.000000,0.000000
50%,7.001205e+09,109.0,0.0,0.0,0.0,197.704000,191.640000,192.080000,176.849000,34.310000,...,0.000000,1.000000,1.000000,1.000000,1.000000,863.000000,0.000000,0.000000,0.000000,0.000000
75%,7.001812e+09,109.0,0.0,0.0,0.0,371.060000,365.344500,369.370500,353.466500,118.740000,...,0.000000,1.000000,1.000000,1.000000,1.000000,1807.500000,0.000000,0.000000,0.000000,0.000000
max,7.002411e+09,109.0,0.0,0.0,0.0,27731.088000,35145.834000,33543.624000,38805.617000,7376.710000,...,49.000000,1.000000,1.000000,1.000000,1.000000,4337.000000,12916.220000,9165.600000,11166.210000,2618.570000


In [7]:
# we can see that there is a lot Date related columns and numerical data related. 
#We will take action in those columns but before let's check the null values %.
round((data.isnull().sum()*100/data.shape[0]),2).sort_values(ascending=False)

count_rech_2g_6             74.85
date_of_last_rech_data_6    74.85
count_rech_3g_6             74.85
av_rech_amt_data_6          74.85
max_rech_data_6             74.85
                            ...  
last_day_rch_amt_6           0.00
last_day_rch_amt_7           0.00
last_day_rch_amt_8           0.00
last_day_rch_amt_9           0.00
mobile_number                0.00
Length: 226, dtype: float64

<B>We can see there are many columns which are showing us more than 74% null values, lets check what are those-

In [8]:
round(data.isnull().sum()/len(data.index),2)[round(data.isnull().sum()/
             len(data.index),2).values>0.10]

date_of_last_rech_data_6    0.75
date_of_last_rech_data_7    0.74
date_of_last_rech_data_8    0.74
date_of_last_rech_data_9    0.74
total_rech_data_6           0.75
total_rech_data_7           0.74
total_rech_data_8           0.74
total_rech_data_9           0.74
max_rech_data_6             0.75
max_rech_data_7             0.74
max_rech_data_8             0.74
max_rech_data_9             0.74
count_rech_2g_6             0.75
count_rech_2g_7             0.74
count_rech_2g_8             0.74
count_rech_2g_9             0.74
count_rech_3g_6             0.75
count_rech_3g_7             0.74
count_rech_3g_8             0.74
count_rech_3g_9             0.74
av_rech_amt_data_6          0.75
av_rech_amt_data_7          0.74
av_rech_amt_data_8          0.74
av_rech_amt_data_9          0.74
arpu_3g_6                   0.75
arpu_3g_7                   0.74
arpu_3g_8                   0.74
arpu_3g_9                   0.74
arpu_2g_6                   0.75
arpu_2g_7                   0.74
arpu_2g_8 

<b>After checking the dataset, it is clear that above mentioned columns are having important role hence we can not drop null values. Hence opting for Imput option.
    But not all data can be treated same as some of them are in Dates and some of them are numbers.

<B> We should also categories the dataset for better understanding and dealing with the records

In [10]:
# Creating column name list by types of columns
id_cols = ['mobile_number', 'circle_id']
date_cols = ['last_date_of_month_6','last_date_of_month_7','last_date_of_month_8','last_date_of_month_9','date_of_last_rech_6','date_of_last_rech_7','date_of_last_rech_8','date_of_last_rech_9','date_of_last_rech_data_6','date_of_last_rech_data_7','date_of_last_rech_data_8','date_of_last_rech_data_9']
cat_cols =  ['night_pck_user_6','night_pck_user_7','night_pck_user_8','night_pck_user_9','fb_user_6','fb_user_7','fb_user_8','fb_user_9']
num_cols = [column for column in data.columns if column not in id_cols + date_cols + cat_cols]


In [11]:
# The number of columns in each listed category 
print("ID cols count in dataset: %d\nDate cols count in dataset:%d\nCategory cols count in dataset:%d\nNumeric cols count in dataset:%d" % (len(id_cols), len(date_cols), len(cat_cols), len(num_cols)))

ID cols count in dataset: 2
Date cols count in dataset:12
Category cols count in dataset:8
Numeric cols count in dataset:204


In [32]:
# It is also observed that the recharge date and the recharge value are missing together which means the customer didn't recharge
data.loc[data['total_rech_data_6'].isnull() & data['date_of_last_rech_data_6'].isnull(), ["total_rech_data_6", "date_of_last_rech_data_6"]].head(20)

,total_rech_data_6,date_of_last_rech_data_6


In [12]:
# As consumers have not recharge their mobile.hence total rechage can be defined as zero. which makes av_rech_amt_data & max_reach_data also zero.
# Now impute missing values with 0,
zero_data = ['total_rech_data_6', 'total_rech_data_7', 'total_rech_data_8', 'total_rech_data_9',
        'av_rech_amt_data_6', 'av_rech_amt_data_7', 'av_rech_amt_data_8', 'av_rech_amt_data_9',
        'max_rech_data_6', 'max_rech_data_7', 'max_rech_data_8', 'max_rech_data_9']

In [13]:
# Now impute missing values with 0
data[zero_data] = data[zero_data].apply(lambda x: x.fillna(0))

In [14]:
#re-check on the dataset shape.
data.shape

(99999, 226)

In [15]:
#Check the data where more null values are present.
round(data.isnull().sum()/len(data.index),2)[round(data.isnull().sum()/
             len(data.index),2).values>0.10]

date_of_last_rech_data_6    0.75
date_of_last_rech_data_7    0.74
date_of_last_rech_data_8    0.74
date_of_last_rech_data_9    0.74
count_rech_2g_6             0.75
count_rech_2g_7             0.74
count_rech_2g_8             0.74
count_rech_2g_9             0.74
count_rech_3g_6             0.75
count_rech_3g_7             0.74
count_rech_3g_8             0.74
count_rech_3g_9             0.74
arpu_3g_6                   0.75
arpu_3g_7                   0.74
arpu_3g_8                   0.74
arpu_3g_9                   0.74
arpu_2g_6                   0.75
arpu_2g_7                   0.74
arpu_2g_8                   0.74
arpu_2g_9                   0.74
night_pck_user_6            0.75
night_pck_user_7            0.74
night_pck_user_8            0.74
night_pck_user_9            0.74
fb_user_6                   0.75
fb_user_7                   0.74
fb_user_8                   0.74
fb_user_9                   0.74
dtype: float64

In [28]:
# Lets move to categorical data.
# cat_cols.describe()
data.describe()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,arpu_9,onnet_mou_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
count,9.999900e+04,99999.0,98981.0,98981.0,98981.0,99999.000000,99999.000000,99999.000000,99999.000000,96062.000000,...,99999.000000,25153.000000,25571.000000,26339.000000,25922.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000
mean,7.001207e+09,109.0,0.0,0.0,0.0,282.987358,278.536648,279.154731,261.645069,132.395875,...,0.084581,0.914404,0.908764,0.890808,0.860968,1219.854749,68.170248,66.839062,60.021204,3.299373
std,6.956694e+05,0.0,0.0,0.0,0.0,328.439770,338.156291,344.474791,341.998630,297.207406,...,0.650457,0.279772,0.287950,0.311885,0.345987,954.733842,267.580450,271.201856,253.938223,32.408353
min,7.000000e+09,109.0,0.0,0.0,0.0,-2258.709000,-2014.045000,-945.808000,-1899.505000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,180.000000,0.000000,0.000000,0.000000,0.000000
25%,7.000606e+09,109.0,0.0,0.0,0.0,93.411500,86.980500,84.126000,62.685000,7.380000,...,0.000000,1.000000,1.000000,1.000000,1.000000,467.000000,0.000000,0.000000,0.000000,0.000000
50%,7.001205e+09,109.0,0.0,0.0,0.0,197.704000,191.640000,192.080000,176.849000,34.310000,...,0.000000,1.000000,1.000000,1.000000,1.000000,863.000000,0.000000,0.000000,0.000000,0.000000
75%,7.001812e+09,109.0,0.0,0.0,0.0,371.060000,365.344500,369.370500,353.466500,118.740000,...,0.000000,1.000000,1.000000,1.000000,1.000000,1807.500000,0.000000,0.000000,0.000000,0.000000
max,7.002411e+09,109.0,0.0,0.0,0.0,27731.088000,35145.834000,33543.624000,38805.617000,7376.710000,...,49.000000,1.000000,1.000000,1.000000,1.000000,4337.000000,12916.220000,9165.600000,11166.210000,2618.570000


In [ ]:
df[cat_cols] = df[cat_cols].apply(lambda x: x.fillna(-1))